### Convert csv to images for data augmentation purposes

In [1]:
import numpy as np
import os
import pandas as pd
from imageio import imwrite

TRAIN_PATH = "data/train"
VALID_PATH = "data/valid"
TEST_PATH = "data/test"
DATASET_PATHS = (TRAIN_PATH, VALID_PATH, TEST_PATH)

IMG_SIZE = 48
OUTPUTS = 7

In [2]:
df = pd.read_csv("data/fer2013.csv")

In [ ]:
# Create directory for training and validation
for dataset_path in DATASET_PATHS:
    # Create directory if it doesn't exist
    if not os.path.exists(dataset_path):
        os.mkdir(dataset_path)

    # For each target that exists
    for target in df["emotion"].unique():
        # Make path for each target
        path_target = os.path.join(dataset_path, str(target))
        # Create directroy if it doesn't exist
        if not os.path.exists(path_target):
            os.mkdir(os.path.join(path_target))

# Iterate through all of the data
for i, row in df.iterrows():
    # Convert image to numpy array and reshape to be 2d
    img = np.array(list(map(int, row["pixels"].split(" "))))
    img = img.reshape((IMG_SIZE, IMG_SIZE)).astype(np.uint8)

    # File name is <number>.png
    fname = str(i)+".png"

    # Make path go to validation directory if less than VALID_PROP
    if row["Usage"] == "Training":
        path = os.path.join(TRAIN_PATH, str(row["emotion"]))
    elif row["Usage"] == "PublicTest":
        path = os.path.join(VALID_PATH, str(row["emotion"]))
    elif row["Usage"] == "PrivateTest":
        path = os.path.join(TEST_PATH, str(row["emotion"]))
    else:
        raise Exception("Invalid Usage: {}".format(row["Usage"]))
        
    # Save image to path
    imwrite(os.path.join(path, fname), img)

### Transfer Learning

In [10]:
from keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from keras.models import Sequential
from keras.optimizers import Adam

def transfer_CNN(base, outputs, lr=0.0005, dropout=0.5, decay=0.001):
    model = Sequential([
        base,
        BatchNormalization(),
        GlobalAveragePooling2D(input_shape=base.output_shape[1:]),
        Dropout(dropout),
        Dense(1024, activation="relu"),
        BatchNormalization(),
        Dropout(dropout),
        Dense(1024, activation="relu"),
        BatchNormalization(),
        Dropout(dropout),
        Dense(outputs, activation="softmax"),
    ])
    opt = Adam(lr=lr, decay=decay)
    model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])
    return model

In [11]:
from keras.preprocessing.image import ImageDataGenerator

# Create ImageDataGenerator for data augmentation
# Rescale image to take values between 0 and 1
# Allow horizontal flipping, x&y shifting, and zooming
train_datagen = ImageDataGenerator(
    rescale=1. / 255,
    horizontal_flip=True,
    width_shift_range=0.10,
    height_shift_range=0.10,
    zoom_range=0.10)

# Create ImageDataGenerator for validation data
# Rescale image to take values between 0 and 1
test_datagen = ImageDataGenerator(rescale=1. / 255)

In [12]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

BATCH_SIZE = 32
EPOCHS = 50

train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True)

validation_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(224, 224),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)


def train(cnn, model_name, epochs=EPOCHS, batch_size=BATCH_SIZE):
    print("Training {}".format(model_name))
    
    callbacks = [
        TensorBoard(), 
        ModelCheckpoint("data/{}.h5".format(model_name), monitor="val_acc", save_best_only=True, save_weights_only=True),
        EarlyStopping(monitor="val_acc", patience=10)
    ]
    
    cnn.model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size + 1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size + 1,
        callbacks=callbacks)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


In [13]:
from keras.applications import InceptionV3

base = InceptionV3(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "Inception")

Training Inception
Epoch 1/50
898/898 [==============================] - 296s 330ms/step - loss: 1.8845 - acc: 0.3865 - val_loss: 1.2793 - val_acc: 0.5191
Epoch 2/50
898/898 [==============================] - 291s 324ms/step - loss: 1.3576 - acc: 0.5214 - val_loss: 1.2363 - val_acc: 0.5400
Epoch 3/50
898/898 [==============================] - 291s 324ms/step - loss: 1.2766 - acc: 0.5473 - val_loss: 1.0830 - val_acc: 0.6024
Epoch 4/50
898/898 [==============================] - 292s 325ms/step - loss: 1.0799 - acc: 0.6048 - val_loss: 1.0333 - val_acc: 0.6180
Epoch 5/50
898/898 [==============================] - 292s 325ms/step - loss: 0.9978 - acc: 0.6327 - val_loss: 1.0045 - val_acc: 0.6319
Epoch 6/50
898/898 [==============================] - 292s 326ms/step - loss: 0.9626 - acc: 0.6533 - val_loss: 1.0087 - val_acc: 0.6252
Epoch 7/50
898/898 [==============================] - 292s 326ms/step - loss: 0.8879 - acc: 0.6773 - val_loss: 0.9392 - val_acc: 0.6584
Epoch 8/50
898/898 [=========

In [14]:
from keras.applications import VGG16

base = VGG16(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "VGG")

Training VGG
Epoch 1/50
898/898 [==============================] - 460s 512ms/step - loss: 2.4379 - acc: 0.1926 - val_loss: 1.8674 - val_acc: 0.2547
Epoch 2/50
898/898 [==============================] - 455s 506ms/step - loss: 2.0606 - acc: 0.2204 - val_loss: 1.8310 - val_acc: 0.2586
Epoch 3/50
898/898 [==============================] - 455s 507ms/step - loss: 1.9369 - acc: 0.2346 - val_loss: 4.0019 - val_acc: 0.1167
Epoch 4/50
898/898 [==============================] - 455s 507ms/step - loss: 1.8525 - acc: 0.2521 - val_loss: 3.1342 - val_acc: 0.1162
Epoch 5/50
898/898 [==============================] - 455s 507ms/step - loss: 1.7994 - acc: 0.2675 - val_loss: 2.0981 - val_acc: 0.1900
Epoch 6/50
898/898 [==============================] - 455s 507ms/step - loss: 1.7563 - acc: 0.2787 - val_loss: 1.6528 - val_acc: 0.3346
Epoch 7/50
898/898 [==============================] - 455s 507ms/step - loss: 1.7048 - acc: 0.3068 - val_loss: 1.9032 - val_acc: 0.2485
Epoch 8/50
898/898 [===============

In [15]:
from keras.applications import ResNet50

base = ResNet50(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "ResNet")

Training ResNet
Epoch 1/50
898/898 [==============================] - 406s 452ms/step - loss: 1.9280 - acc: 0.3797 - val_loss: 1.3266 - val_acc: 0.4968
Epoch 2/50
898/898 [==============================] - 402s 447ms/step - loss: 1.3123 - acc: 0.5181 - val_loss: 1.1516 - val_acc: 0.5673
Epoch 3/50
898/898 [==============================] - 410s 456ms/step - loss: 1.1524 - acc: 0.5763 - val_loss: 1.1770 - val_acc: 0.5475
Epoch 4/50
898/898 [==============================] - 424s 472ms/step - loss: 1.0824 - acc: 0.6008 - val_loss: 1.3988 - val_acc: 0.5258
Epoch 5/50
898/898 [==============================] - 423s 471ms/step - loss: 1.0423 - acc: 0.6191 - val_loss: 1.0276 - val_acc: 0.6166
Epoch 6/50
898/898 [==============================] - 454s 505ms/step - loss: 0.9852 - acc: 0.6404 - val_loss: 0.9857 - val_acc: 0.6356
Epoch 7/50
898/898 [==============================] - 457s 509ms/step - loss: 0.9393 - acc: 0.6558 - val_loss: 0.9753 - val_acc: 0.6319
Epoch 8/50
898/898 [============

In [16]:
from keras.applications import DenseNet121

base = DenseNet121(include_top=False, weights="imagenet")
model = transfer_CNN(base, OUTPUTS)
train(model, "DenseNet")

30015488/30011760 [==============================] - 6s 0us/step
Training DenseNet
Epoch 1/50


ResourceExhaustedError: OOM when allocating tensor with shape[32,736,7,7] and type float on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[Node: densenet121/conv5_block8_0_bn/FusedBatchNorm = FusedBatchNorm[T=DT_FLOAT, data_format="NHWC", epsilon=1.001e-05, is_training=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](densenet121/conv5_block7_concat/concat, conv5_block8_0_bn/gamma/read, conv5_block8_0_bn/beta/read, densenet121/conv1/bn/Const, densenet121/conv1/bn/Const)]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.

	 [[Node: metrics_3/acc/Mean/_26919 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/device:CPU:0", send_device="/job:localhost/replica:0/task:0/device:GPU:0", send_device_incarnation=1, tensor_name="edge_43854_metrics_3/acc/Mean", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/device:C

### Construct CNN architectures

In [2]:
from keras.applications.densenet import dense_block, transition_block
from keras.applications.resnet50 import identity_block, conv_block
from keras.layers import Activation, AveragePooling2D, BatchNormalization, concatenate, Conv2D
from keras.layers import Dense, Dropout, Flatten, GlobalAveragePooling2D, Input, MaxPooling2D, ZeroPadding2D
from keras.models import Model
from keras.optimizers import Adam

class CNN:
    """Abstract base class for all CNNS"""
    
    model = None

    def __init__(self, lr=0.001, decay=0):
        # Use Adam for optimizer with configurable lr and decay
        opt = Adam(lr=lr, decay=decay)
        # Compile model and use crossentropy with accuracy as a metric
        self.model.compile(optimizer=opt, loss="categorical_crossentropy", metrics=["accuracy"])

    def Conv2D_bn(self, x, nb_filter, filter_size, strides=(1, 1), padding="same"):
        """2D Convolutional Batch Normalization block"""
        # Convolutional layer with configurable filters, strides, and padding
        x = Conv2D(nb_filter, (filter_size, filter_size), strides=strides, padding=padding)(x)
        # Use batch norm to normalize the inputs for the next layer
        x = BatchNormalization()(x)
        # Use ReLU as nonlinearity
        return Activation("relu")(x)


class VGG(CNN):
    """Model based on VGG"""
    
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        # Create vgg_block with 2 convolutions with 64 filters
        x = self.vgg_block(img_input, 2, 64)
        # Create vgg_block with 2 convolutions with 128 filters
        x = self.vgg_block(x, 2, 128)
        # Create vgg_block with 3 convolutions with 256 filters
        x = self.vgg_block(x, 3, 256)
        # Create vgg_block with 3 convolutions with 512 filters
        x = self.vgg_block(x, 3, 512)
        # Flatten convolutional output to fit with following dense layer
        x = Flatten()(x)
        # Create dense batch norm layer with 1024 units
        x = self.Dense_bn(x, 1024)
        # Use dropout with configurable p
        x = Dropout(dropout)(x)
        # Create dense batch norm layer with 1024 units
        x = self.Dense_bn(x, 1024)
        # Use dropout with configurable p
        x = Dropout(dropout)(x)
        # Dense prediction layer with softmax
        predictions = Dense(outputs, activation="softmax")(x)
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

    def vgg_block(self, x, nb_conv, nb_filters, filter_size=3):
        """VGG Block will place nb_conv convolutional layers each with nb_filters filters.
           It is followed by a max pooling layer that halves the height and width.
        """
        # Create nb_conv convolutions
        for i in range(nb_conv):
            x = self.Conv2D_bn(x, nb_filters, filter_size)
        # Use 2x2 max pooling with stride 2x2
        return MaxPooling2D((2, 2), strides=(2, 2))(x)
            
    def Dense_bn(self, x, units):
        """Dense layer followed by Batch Normalization and ReLU"""
        # Create dense layer with configurable units
        x = Dense(units)(x)
        # Use batch norm to normalize the inputs for the next layer
        x = BatchNormalization()(x)
        # Use ReLU as nonlinearity
        return Activation("relu")(x)


class Inception_FCN(CNN):
    """Model that utilizes Google's Inception block"""
    
    def __init__(self, outputs, input_shape, lr=0.001, decay=0, dropout=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        # Create 4 inception blocks to reduce activation map to height and width of 3x3
        x = self.inception_block(img_input)
        x = self.inception_block(x)
        x = self.inception_block(x)
        x = self.inception_block(x)
        # Use dropout to reduce overfitting
        x = Dropout(dropout)(x)
        # Use convolutional layer with filters same as classes to produce outputs
        x = Conv2D(outputs, (3, 3), padding="same")(x)
        # Use GlobalAveragePooling to convert 3x3x3 activations to 1x3 for output
        x = GlobalAveragePooling2D()(x)
        # Use softmax layer
        predictions = Activation("softmax")(x)
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

    def inception_block(self, x):
        """Inception block from GoogleNet paper.
           Uses filters of different sizes and combines their activations,
           which helps it look for various features in an image.
        """
        # Do convolution with 1x1 filter stride 2x2
        branch1x1 = self.Conv2D_bn(x, 64, 1, strides=(2, 2))

        # Do convolution with 1x1 filter
        branch5x5 = self.Conv2D_bn(x, 48, 1)
        # Do convolution with 5x5 filter stride 2x2
        branch5x5 = self.Conv2D_bn(branch5x5, 64, 5, strides=(2, 2))

        # Do convolution with 1x1 filter
        branch3x3dbl = self.Conv2D_bn(x, 64, 1)
        # Do convolution with 3x3 filter
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3)
        # Do convolution with 3x3 filter stride 2x2
        branch3x3dbl = self.Conv2D_bn(branch3x3dbl, 96, 3, strides=(2, 2))

        # Do 3x3 averagepooling with stride 2x2
        branch_pool = AveragePooling2D((3, 3), strides=(2, 2), padding="same")(x)
        # Do convolution with 1x1 filter
        branch_pool = self.Conv2D_bn(branch_pool, 64, 1)
        
        # Concatenate resulting activations along the filters axis
        return concatenate([branch1x1, branch5x5, branch3x3dbl, branch_pool], axis=-1)


class ResNet(CNN):
    """Model based on ResNet by using shortcut connections"""
    def __init__(self, outputs, input_shape, lr=0.001, decay=0):
        # Input layer takes image shape
        img_input = Input(shape=input_shape)
        
        # Shortcut connections allow gradients to propagate more easily through deep networks
        # conv_block has a convolutional layer at the shortcut connection
        # identity_block has no convolutional layer at the shortcut connection
        
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = conv_block(img_input, 3, [32, 32, 128], stage=1, block="a", strides=(1, 1))
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="b")
        # Do 3 3x3 convolutional layers of 32, 32, 128 filters
        x = identity_block(x, 3, [32, 32, 128], stage=1, block="c")

        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = conv_block(x, 3, [64, 64, 256], stage=2, block="a")
        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="b")
        # Do 3 3x3 convolutional layers of 64, 64, 256 filters
        x = identity_block(x, 3, [64, 64, 256], stage=2, block="c")

        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = conv_block(x, 3, [128, 128, 512], stage=3, block="a")
        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="b")
        # Do 3 3x3 convolutional layers of 128, 128, 512 filters
        x = identity_block(x, 3, [128, 128, 512], stage=3, block="c")

        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = conv_block(x, 3, [256, 256, 1024], stage=4, block="a")
        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="b")
        # Do 3 3x3 convolutional layers of 256, 256, 1024 filters
        x = identity_block(x, 3, [256, 256, 1024], stage=4, block="c")

        # Do GlobalAveragePooling to reduce the 6x6x1024 output to 1x1024
        x = GlobalAveragePooling2D(name="global_avg_pool")(x)
        # Use dense layer to map to number of outputs followed by softmax
        predictions = Dense(outputs, activation="softmax")(x)

        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)
        
        
class DenseNet(CNN):
    """Model based on DenseNet"""
    def __init__(self, outputs, input_shape, lr=0.01, decay=0, dropout=0):
        self.blocks = [3, 6, 12, 8]
        img_input = Input(shape=input_shape)
        x = dense_block(img_input, self.blocks[0], name="conv1")
        x = transition_block(x, 0.5, name="pool1")
        x = dense_block(x, self.blocks[1], name="conv2")
        x = transition_block(x, 0.5, name="pool2")
        x = dense_block(x, self.blocks[2], name="conv3")
        x = transition_block(x, 0.5, name="pool3")
        x = dense_block(x, self.blocks[3], name="conv4")
        
        x = BatchNormalization()(x)
        x = GlobalAveragePooling2D(name="global_avg_pool")(x)
        x = Dropout(dropout)(x)
        predictions = Dense(outputs, activation="softmax")(x)
        
        # Create Keras functional Model
        self.model = Model(inputs=img_input, outputs=predictions)
        # Use base class to compile
        super().__init__(lr, decay)

/home/dat/.Envs/tf/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

BATCH_SIZE = 128
EPOCHS = 50

# Apply generator to training data
train_generator = train_datagen.flow_from_directory(
    "data/train",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=True)

# Apply generator to validation data
validation_generator = test_datagen.flow_from_directory(
    "data/valid",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)


def train(cnn, epochs=EPOCHS, batch_size=BATCH_SIZE):
    model_name = type(cnn).__name__
    print("Training {}".format(model_name))
    # Create callbacks for checkpointing model progress
    callbacks = [
        TensorBoard(), 
        ModelCheckpoint("data/{}.h5".format(model_name), monitor="val_acc", save_best_only=True, save_weights_only=True),
        EarlyStopping(monitor="val_acc", patience=10)
    ]
    # Train each cnn
    cnn.model.fit_generator(
        train_generator,
        steps_per_epoch=train_generator.n // batch_size + 1,
        epochs=epochs,
        validation_data=validation_generator,
        validation_steps=validation_generator.n // batch_size + 1,
        callbacks=callbacks)

Found 28709 images belonging to 7 classes.
Found 3589 images belonging to 7 classes.


### Train models for ensemble

In [20]:
vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.5, decay=0.0001)
train(vgg)

Training VGG
Epoch 1/50
225/225 [==============================] - 38s 171ms/step - loss: 1.9439 - acc: 0.2562 - val_loss: 3.8721 - val_acc: 0.2449
Epoch 2/50
225/225 [==============================] - 36s 159ms/step - loss: 1.6385 - acc: 0.3992 - val_loss: 13.6105 - val_acc: 0.1323
Epoch 3/50
225/225 [==============================] - 36s 159ms/step - loss: 1.4695 - acc: 0.4641 - val_loss: 1.4582 - val_acc: 0.4152
Epoch 4/50
225/225 [==============================] - 36s 159ms/step - loss: 1.2909 - acc: 0.5115 - val_loss: 1.4263 - val_acc: 0.4656
Epoch 5/50
225/225 [==============================] - 36s 160ms/step - loss: 1.2082 - acc: 0.5425 - val_loss: 1.4442 - val_acc: 0.4748
Epoch 6/50
225/225 [==============================] - 36s 160ms/step - loss: 1.1453 - acc: 0.5701 - val_loss: 1.2251 - val_acc: 0.5483
Epoch 7/50
225/225 [==============================] - 36s 160ms/step - loss: 1.1035 - acc: 0.5853 - val_loss: 1.1476 - val_acc: 0.5614
Epoch 8/50
225/225 [=====================

In [4]:
inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, dropout=0.4, decay=0.001)
train(inception)

Training Inception_FCN
Epoch 1/50
225/225 [==============================] - 49s 219ms/step - loss: 1.7306 - acc: 0.3032 - val_loss: 2.8068 - val_acc: 0.2095
Epoch 2/50
225/225 [==============================] - 46s 203ms/step - loss: 1.5386 - acc: 0.4011 - val_loss: 1.7949 - val_acc: 0.3686
Epoch 3/50
225/225 [==============================] - 46s 204ms/step - loss: 1.3851 - acc: 0.4686 - val_loss: 1.4982 - val_acc: 0.4405
Epoch 4/50
225/225 [==============================] - 44s 195ms/step - loss: 1.2955 - acc: 0.5048 - val_loss: 1.2711 - val_acc: 0.5141
Epoch 5/50
225/225 [==============================] - 48s 211ms/step - loss: 1.2252 - acc: 0.5343 - val_loss: 1.2837 - val_acc: 0.5155
Epoch 6/50
225/225 [==============================] - 47s 208ms/step - loss: 1.1778 - acc: 0.5504 - val_loss: 1.1693 - val_acc: 0.5475
Epoch 7/50
225/225 [==============================] - 47s 208ms/step - loss: 1.1368 - acc: 0.5686 - val_loss: 1.1994 - val_acc: 0.5350
Epoch 8/50
225/225 [============

In [5]:
resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, decay=0.001)
train(resnet)

Training ResNet
Epoch 1/50
225/225 [==============================] - 100s 446ms/step - loss: 1.8238 - acc: 0.2701 - val_loss: 1.9963 - val_acc: 0.3076
Epoch 2/50
225/225 [==============================] - 100s 445ms/step - loss: 1.5536 - acc: 0.3944 - val_loss: 2.3143 - val_acc: 0.3583
Epoch 3/50
225/225 [==============================] - 98s 437ms/step - loss: 1.3888 - acc: 0.4677 - val_loss: 1.9490 - val_acc: 0.4074
Epoch 4/50
225/225 [==============================] - 99s 439ms/step - loss: 1.2779 - acc: 0.5083 - val_loss: 1.3719 - val_acc: 0.4667
Epoch 5/50
225/225 [==============================] - 97s 432ms/step - loss: 1.2125 - acc: 0.5375 - val_loss: 1.4945 - val_acc: 0.4935
Epoch 6/50
225/225 [==============================] - 98s 434ms/step - loss: 1.1550 - acc: 0.5621 - val_loss: 1.3990 - val_acc: 0.4728
Epoch 7/50
225/225 [==============================] - 100s 445ms/step - loss: 1.1164 - acc: 0.5787 - val_loss: 1.2807 - val_acc: 0.5283
Epoch 8/50
225/225 [================

In [6]:
densenet = DenseNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0.001, decay=0.001, dropout=0.2)
train(densenet)

Training DenseNet
Epoch 1/50
225/225 [==============================] - 105s 467ms/step - loss: 1.8353 - acc: 0.2718 - val_loss: 8.3087 - val_acc: 0.2588
Epoch 2/50
225/225 [==============================] - 101s 450ms/step - loss: 1.5881 - acc: 0.3786 - val_loss: 3.1895 - val_acc: 0.1808
Epoch 3/50
225/225 [==============================] - 103s 457ms/step - loss: 1.4045 - acc: 0.4541 - val_loss: 1.5537 - val_acc: 0.4093
Epoch 4/50
225/225 [==============================] - 103s 458ms/step - loss: 1.2916 - acc: 0.5046 - val_loss: 1.6289 - val_acc: 0.3820
Epoch 5/50
225/225 [==============================] - 103s 459ms/step - loss: 1.1942 - acc: 0.5437 - val_loss: 1.2978 - val_acc: 0.4948
Epoch 6/50
225/225 [==============================] - 101s 450ms/step - loss: 1.1096 - acc: 0.5786 - val_loss: 1.3551 - val_acc: 0.5068
Epoch 7/50
225/225 [==============================] - 103s 458ms/step - loss: 1.0561 - acc: 0.6035 - val_loss: 1.2366 - val_acc: 0.5347
Epoch 8/50
225/225 [==========

### Check ensemble validation accuracy and produce outputs

In [15]:
BATCH_SIZE = 256

def predict_ensemble(models, imgs, weights):
    """Average the predictions of all models to get ensemble prediction.
    """
    batch_size = imgs.shape[0]
    # Create empty array for model predictions
    predictions = np.zeros((len(models), batch_size, OUTPUTS))
    # Predict the class of the image for each model
    for i in range(len(models)):
        predictions[i] = models[i].predict(imgs)
    # Return the weighted average prediction of all models
    return np.argmax(np.average(predictions, axis=0, weights=weights), axis=-1).squeeze()


# Load 3 trained models
vgg = VGG(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
vgg.model.load_weights("data/VGG.h5")
inception = Inception_FCN(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, dropout=0, decay=0)
inception.model.load_weights("data/Inception_FCN.h5")
resnet = ResNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, decay=0)
resnet.model.load_weights("data/ResNet.h5")
densenet = DenseNet(OUTPUTS, (IMG_SIZE, IMG_SIZE, 1), lr=0, decay=0)
densenet.model.load_weights("data/DenseNet.h5")
models = [vgg.model, inception.model, resnet.model, densenet.model]
weights = [0.2667, 0.2, 0.2667, 0.2667]


# Apply generator to validation data
validation_generator = test_datagen.flow_from_directory(
    "data/test",
    target_size=(IMG_SIZE, IMG_SIZE),
    color_mode="grayscale",
    batch_size=BATCH_SIZE,
    class_mode="categorical",
    shuffle=False)

# Create numpy array for predictions
predictions = np.zeros(validation_generator.n)
# Create numpy array for targets
targets = np.zeros(validation_generator.n)

# Iterate through all validation data
for i, (imgs, target) in enumerate(validation_generator):
    if i >= validation_generator.n // BATCH_SIZE + 1:
        break
    # Get prediction of ensemble
    predictions[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = predict_ensemble(models, imgs, weights)
    # Get target
    targets[i*BATCH_SIZE:(i+1)*BATCH_SIZE] = np.argmax(target, axis=1)
accuracy = np.mean(targets == predictions)
print("Accuracy: {}".format(accuracy))

Found 3589 images belonging to 7 classes.
Accuracy: 0.7160769016439119


VGG: 0.6760

Inception: 0.6517

ResNet: 0.6617

DenseNet: 0.6726

VGG & DenseNet: 0.6985

VGG & Inception & DenseNet: 0.7021

VGG & ResNet & DenseNet: 0.7058

VGG & Inception & ResNet & DenseNet: 0.7063


weights = [0.2667, 0.2, 0.2667, 0.2667] 0.7080